<a href="https://colab.research.google.com/github/Ranjanunicode/transformerBHU/blob/main/attn_scratch_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn

In [ ]:
def softmax(z):
  z=np.exp(z-z.max(axis=-1, keepdims=True))
  return z / z.sum(axis=-1, keepdims=True)

def self_attention(X, mask, W_KQV, W_out):
  K,Q,V = np.split(X@W_KQV, 3, axis=1)
  attn = softmax(K@Q.T / np.sqrt(X.shape[1]) + mask)
  return attn@V@W_out, attn


In [ ]:
T, d = 100, 64
attn = nn.MultiheadAttention(d, 1, bias=False, batch_first=True)
M = torch.triu(-float("inf")*torch.ones(T,T),1)
X = torch.randn(1,T,d)
Y_, A_ = attn(X, X, X, attn_mask=M)

In [ ]:
attn.out_proj.weight.shape

torch.Size([64, 64])

In [ ]:
Y, A = self_attention(X[0].numpy(),
                      M.numpy(),
                      attn.in_proj_weight.detach().numpy().T,
                      attn.out_proj.weight.detach().numpy().T)

In [ ]:
np.linalg.norm(A - A_[0].detach().numpy())

2.2423544e-07

Minibatching

In [ ]:
C = np.random.randn(5,4,10,3)
D = np.random.randn(3,6)
(C@D).shape

(5, 4, 10, 6)

In [ ]:
# (C.reshape(-1,3) @ D).reshape(5,4,10,6)
def self_attention(X, mask, W_KQV, W_out):
  K,Q,V = np.split(X@W_KQV, 3, axis=-1)
  attn = softmax(K@Q.swapaxes(-1,-2) / np.sqrt(X.shape[-1]) + mask)
  return attn@V@W_out, attn

In [ ]:
B, T, d = 50, 100, 64
X = torch.randn(B,T,d)
M = torch.triu(-float("inf")*torch.ones(T,T),1)
Y_, A_ = attn(X, X, X, attn_mask=M)

In [ ]:
Y, A = self_attention(X.numpy(),
                      M.numpy(),
                      attn.in_proj_weight.detach().numpy().T,
                      attn.out_proj.weight.detach().numpy().T)

In [ ]:
np.linalg.norm(A - A_.detach().numpy())

1.5895062e-06

Multihead attention